# Preview Parquet – NYC Taxi TLC (SPIKE Big Data)

Objectif :
- Prévisualiser un fichier Parquet volumineux
- Identifier le schéma, les colonnes chiffrées et la volumétrie
- Qualifier la source pour un projet Big Data

Aucun traitement, aucune transformation métier.

## Installation des dépendances 

Si nécessaire, exécuter dans un terminal ou ici :
`pip install pandas pyarrow`

In [9]:
!pip install pandas pyarrow

## Chargement du fichier Parquet

Adapter le chemin vers le fichier téléchargé depuis NYC TLC.

In [10]:
import pyarrow.parquet as pq

path = "yellow_tripdata_2025-01.parquet"  # à adapter
table = pq.read_table(path)

print("Fichier chargé")

Fichier chargé


## Schéma du dataset

In [11]:
table.schema

VendorID: int32
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: int64
trip_distance: double
RatecodeID: int64
store_and_fwd_flag: large_string
PULocationID: int32
DOLocationID: int32
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
Airport_fee: double
cbd_congestion_fee: double

## Informations de volumétrie

In [12]:
import os
import pyarrow.parquet as pq

path = "yellow_tripdata_2025-01.parquet"  # adapte si besoin

table = pq.read_table(path)

file_size_bytes = os.path.getsize(path)
file_size_mb = file_size_bytes / (1024 * 1024)
file_size_gb = file_size_mb / 1024

print("Nombre de lignes :", table.num_rows)
print("Nombre de colonnes :", table.num_columns)
print(f"Taille du fichier : {file_size_mb:.2f} MB ({file_size_gb:.2f} GB)")

Nombre de lignes : 3475226
Nombre de colonnes : 20
Taille du fichier : 56.42 MB (0.06 GB)


## Liste des colonnes

In [13]:
table.column_names

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee',
 'cbd_congestion_fee']

## Aperçu des premières lignes

Extraction d’un petit échantillon sans charger tout le dataset.

In [14]:
table.slice(0, 5).to_pandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1,1.60,1,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1,0.50,1,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1,0.60,1,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3,0.52,1,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3,0.66,1,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.0


## Collecte et stockage des données (SPIKE d’architecture)

### Collecte des données
- Source : NYC Taxi & Limousine Commission – Trip Record Data
- Mode d’accès : site officiel de téléchargement
- Données proposées sous forme de fichiers mensuels
  - Exemple : `yellow_tripdata_YYYY-MM.parquet`
- Obtention des données :
  - téléchargement direct (open data)
  - solution légale et réaliste
- Processus de collecte :
  - sélection d’un ou plusieurs mois
  - téléchargement automatisable (script)
  - processus reproductible et extensible
- La structure des fichiers étant stable, la volumétrie peut être augmentée
  simplement en ajoutant de nouveaux fichiers mensuels

### Stockage des données
- Mise en place d’un Data Lake logique
- Zone **RAW** :
  - stockage des fichiers Parquet tels quels
  - aucune transformation appliquée
  - conservation de la donnée brute
- Zone **CURATED** :
  - stockage des résultats des réductions Spark
  - données agrégées et exploitables
  - format adapté à la réutilisation (Parquet / table)


## Qualification initiale de la source de données (SPIKE)

### Source
- Nom : NYC Taxi & Limousine Commission – Trip Record Data (Yellow Taxi)
- Type : Open Data
- Format : Parquet
- Accès : Téléchargement mensuel (fichiers indépendants)

### Structure des données
- Données structurées
- 1 ligne = 1 trajet
- Schéma stable d’un fichier mensuel à l’autre
- Colonnes majoritairement numériques (montants, distances, compteurs)
- Présence de quelques colonnes non numériques (dates, indicateurs)

### Volumétrie
- Environ 3,5 millions de lignes pour un fichier mensuel
- 20 colonnes par fichier
- Volumétrie extensible par agrégation de plusieurs mois
  - 3 mois ≈ 10 millions de lignes
  - 6 mois ≈ 20 millions de lignes

### Vélocité
- Données publiées régulièrement (mensuel)
- Historisation possible sur plusieurs années

### Variété
- Données hétérogènes :
  - métriques numériques
  - identifiants
  - informations temporelles
- Données répétées par identifiants (VendorID, zones)

### Intérêt Big Data
- Volumétrie élevée justifiant un traitement distribué
- Opérations d’agrégation nécessaires (groupBy)
- Redistribution des données (shuffle) lors des traitements
- Source adaptée à l’utilisation d’Apache Spark


## Hypothèses de réutilisation des données (SPIKE)

### Principe général
- La source NYC Taxi n’est pas alignée métier avec le modèle e-commerce.
- Elle est utilisée comme **source volumineuse à détourner**.
- L’objectif est de réorganiser certaines colonnes numériques afin de simuler
  des indicateurs environnementaux exploitables.

### Hypothèses d’adaptation des colonnes

#### Colonnes conservées / détournées
- `VendorID`
  - Rôle : identifiant répété
  - Hypothèse : utilisé comme **identifiant produit simulé** (`id_product_ref`)
- `PULocationID` / `DOLocationID`
  - Rôle : zones géographiques
  - Hypothèse : utilisées comme **identifiant d’entrepôt** (`id_warehouse`)
- `fare_amount`, `total_amount`, `trip_distance`
  - Rôle : métriques numériques continues
  - Hypothèse :
    - `fare_amount` → `ec_process`
    - `trip_distance` ou `total_amount` → `ec_transport`

#### Colonnes ignorées (non réutilisées)
- Colonnes temporelles (`pickup_datetime`, `dropoff_datetime`)
- Colonnes métier spécifiques au taxi (`payment_type`, `store_and_fwd_flag`)
- Surcharges spécifiques (taxes détaillées non nécessaires au modèle final)

### Colonnes ajoutées / simulées
- `size`
  - Taille produit simulée (XS, S, M, L)
  - Ajoutée via table auxiliaire ou génération contrôlée
- `stock_qty`
  - Quantité simulée par produit et entrepôt
  - Utilisée pour crédibiliser le modèle e-commerce

### Logique de transformation envisagée
- Réduction des données par agrégation Spark
- Regroupement par identifiant produit simulé
- Calculs distribués sur un volume important
- Production d’une table métier exploitable par les modules suivants
